# PAGE hm3 analysis

### Step 2: Calculate GRM
Calculate GRM matrices
```

### Step 3: estimate parameters
```bash
for n_indiv in 5000 10000 20000; do
    for anc_prop in 0.2 0.5; do
        dset_prefix=n_indiv_${n_indiv}_anc_prop_${anc_prop}
        for cor in 0.5 0.8 1.0; do
            qsub estimate_HE.sh ${dset_prefix} ${cor}
        done
    done
done
```

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black

In [2]:
import xarray as xr
import numpy as np
import admix
import matplotlib.pyplot as plt
import dask
import dask.array as da
from tqdm import tqdm
import admix_genet_cor
import pandas as pd

In [3]:
dset = admix_genet_cor.load_page_hm3()

100%|██████████| 22/22 [00:04<00:00,  4.84it/s]


In [5]:
# a = np.load("out/OLD_admix_grm/K1.all.npy")[0:100, :] + np.load("out/OLD_admix_grm/K2.all.npy")[0:100, :]
# b = np.load("out/admix-grm/hm3.uniform.all.A1.npy")[0: 100, :]
# c = np.load("out/admix-grm/imputed.mafukb.all.A1.npy")[0:100, :]

In [6]:
# K1 = np.load("out/OLD_admix_grm/K1.all.npy")
# K2 = np.load("out/OLD_admix_grm/K2.all.npy")
# K12 = np.load("out/OLD_admix_grm/K12.all.npy")
# dset["A1"] = (("indiv", "indiv"), np.load("out/admix-grm/imputed.mafukb.all.A1.npy"))
# dset["A2"] = (("indiv", "indiv"), np.load("out/admix-grm/imputed.mafukb.all.A2.npy"))
# del K1, K2, K12

dset["A1"] = (("indiv", "indiv"), np.load("out/admix-grm/hm3.mafukb.all.A1.npy"))
dset["A2"] = (("indiv", "indiv"), np.load("out/admix-grm/hm3.mafukb.all.A2.npy"))

In [7]:
trait_list = [
    # Inflammtory traits
    "crp",
    "total_wbc_cnt",
    "mean_corp_hgb_conc",
    "platelet_cnt",
    # lipid traits
    "hdl",
    "ldl",
    "triglycerides",
    "total_cholesterol",
    # lifestyle traits
    "cigs_per_day_excl_nonsmk_updated",
    "coffee_cup_day",
    # glycemic traits
    "a1c",
    "insulin",
    "glucose",
    "t2d_status",
    # electrocardiogram traits
    "qt_interval",
    "qrs_interval",
    "pr_interval",
    # blood pressure traits
    "systolic_bp",
    "diastolic_bp",
    "hypertension",
    # anthropometric traits
    "waist_hip_ratio",
    "height",
    "bmi",
    # kidney traits
    "egfrckdepi",
]

In [ ]:
dict_result = {"trait": [], 
             "n_indiv": [], 
             "est": [],
             "est_var": []}

for trait in tqdm(trait_list):
    dset_cor = dset.isel(indiv=~np.isnan(dset[f"{trait}"].values))
    study_dummies = pd.get_dummies(dset_cor["study"], drop_first=True)
    for c in study_dummies:
        dset_cor[f"study_dummy_{c}"] = ("indiv", study_dummies[c])
    study_dummy_cols = [f"study_dummy_{c}" for c in study_dummies]
    
    pheno = dset_cor[f"{trait}"].values
    pheno = admix.data.quantile_normalize(pheno)
    
    cov_cols=["age", "sex"] + study_dummy_cols + [f"geno_EV{i}" for i in range(1, 11)]
    cov = np.c_[np.ones((dset_cor.dims["indiv"], 1)), np.vstack([dset_cor[col].data for col in cov_cols]).T]
    normalized_cov = np.array(cov)
    for i in range(1, cov.shape[1]):
#         normalized_cov[:, i] = admix.data.quantile_normalize(normalized_cov[:, i])
        normalized_cov[:, i] = (normalized_cov[:, i] - normalized_cov[:, i].mean()) / normalized_cov[:, i].std()
    est, est_var = admix_genet_cor.estimate_genetic_cor(A1=dset_cor["A1"].data, A2=dset_cor["A2"].data, pheno=pheno, cov=normalized_cov, compute_varcov=True)[0]
    
#     est, est_var = admix.estimate.admix_gen_cor(dset=dset_cor, 
#                                                 pheno=pheno, 
#                                                 cov_cols=["age", "sex"] + study_dummy_cols + [f"geno_EV{i}" for i in range(1, 11)])[0]
    print('--------------')
    print(f"{trait} (N={dset_cor.dims['indiv']})")
    print(est)
    print(est_var)
    
    dict_result["trait"].append(trait)
    dict_result["n_indiv"].append(dset_cor.dims['indiv'])
    dict_result["est"].append(est)
    dict_result["est_var"].append(est_var)

  0%|          | 0/24 [00:00<?, ?it/s]/u/project/pasaniuc/kangchen/software/miniconda3/lib/python3.8/site-packages/xarray/core/indexing.py:1226: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]

  4%|▍         | 1/24 [02:50<1:05:11, 170.08s/it]

--------------
crp (N=8521)
[0.88029924 0.83271288 0.69111675]
[[ 0.02788207  0.03028527 -0.00789622]
 [ 0.03028527  0.03522145 -0.0084219 ]
 [-0.00789622 -0.0084219   0.00246023]]



  8%|▊         | 2/24 [06:01<1:06:55, 182.54s/it]

--------------
total_wbc_cnt (N=8889)
[1.74641433 0.68560884 0.30634003]
[[ 0.13933774  0.13696613 -0.04127721]
 [ 0.13696613  0.19858521 -0.03718428]
 [-0.04127721 -0.03718428  0.01261213]]



 12%|█▎        | 3/24 [06:18<37:31, 107.24s/it]  

...


 50%|█████     | 12/24 [40:31<36:16, 181.38s/it]

--------------
insulin (N=7927)
[0.59421066 0.61952427 0.79864545]
[[ 0.02678899  0.02968644 -0.00764744]
 [ 0.02968644  0.03379579 -0.00839804]
 [-0.00764744 -0.00839804  0.00242914]]



 54%|█████▍    | 13/24 [44:59<38:05, 207.78s/it]

--------------
glucose (N=9937)
[0.21655688 0.19201887 0.8281267 ]
[[ 0.01390763  0.01541751 -0.00404126]
 [ 0.01541751  0.0177312  -0.00443304]
 [-0.00404126 -0.00443304  0.0013393 ]]



  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
df_rls = dict()
for name in ["trait", "n_indiv"]:
    df_rls[name] = dict_result[name]

for i, name in enumerate(["sigma_g", "gamma", "sigma_e"]):
    df_rls[name] = [f"{e[i]:.2f} ({np.sqrt(v[i, i]):.2f})" for e, v in zip(dict_result["est"], dict_result["est_var"])]
df_rls = pd.DataFrame(df_rls)

In [ ]:
df_rls

In [ ]:
df_rls.to_csv("result/page.csv", index=False)